In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim
from model import*
from residual_block import*

In [3]:
model = ResNet(ResidualBlock,num_classes=10)

In [4]:
model 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (g): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (g): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, mome

In [5]:
#Mnist dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


In [7]:
#import imagenet data
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [00:13<00:00, 12990248.05it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [9]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (g): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (g): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, mome

In [10]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [11]:
from tqdm.notebook import tqdm

In [19]:
epochs = 50
loss = []
acc = []
for i in tqdm(range(epochs)):
    model.train()
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {i+1}/{epochs}, Loss: {loss.item()}')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50, Loss: 0.2558765709400177
Epoch 2/50, Loss: 0.27656570076942444
Epoch 3/50, Loss: 0.17238730192184448
Epoch 4/50, Loss: 0.26674333214759827
Epoch 5/50, Loss: 0.09386350959539413
Epoch 6/50, Loss: 0.05812779814004898
Epoch 7/50, Loss: 0.1034557893872261
Epoch 8/50, Loss: 0.03078206442296505
Epoch 9/50, Loss: 0.062257107347249985
Epoch 10/50, Loss: 0.0303523950278759
Epoch 11/50, Loss: 0.13783450424671173
Epoch 12/50, Loss: 0.14180880784988403
Epoch 13/50, Loss: 0.05600421875715256
Epoch 14/50, Loss: 0.1642863005399704
Epoch 15/50, Loss: 0.03979168087244034
Epoch 16/50, Loss: 0.026945948600769043
Epoch 17/50, Loss: 0.045004770159721375
Epoch 18/50, Loss: 0.06650946289300919
Epoch 19/50, Loss: 0.05691932514309883
Epoch 20/50, Loss: 0.11769209057092667
Epoch 21/50, Loss: 0.07151149213314056
Epoch 22/50, Loss: 0.2300487458705902
Epoch 23/50, Loss: 0.04357630014419556
Epoch 24/50, Loss: 0.12816467881202698
Epoch 25/50, Loss: 0.12073659151792526
Epoch 26/50, Loss: 0.082812190055847

In [20]:
#evaluation
from torch.autograd import Variable
model.eval();
testloss = 0.
testacc = 0.
for (img,label) in test_loader:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    out = model(img)
    loss = criteria(out,label)
    testloss += loss.item()
    _,pred = torch.max(out,1)
    num_correct = (pred == label).sum()
    testacc += num_correct.item()

testloss /= len(test_loader.dataset)
testacc /= len(test_loader.dataset)
print(f'Test Loss: {testloss}, Test Accuracy: {testacc}')

Test Loss: 0.026017068409919738, Test Accuracy: 0.6618
